In [1]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

from pycaret.regression import *
from pycaret.regression import setup, compare_models

In [10]:
df = pd.read_csv("../data/clean/df_encoded.csv")
print(df.shape)

(14632, 51)


In [11]:
df.isna().sum().sum()

4426

In [12]:
df = pd.read_csv("../data/clean/df_encoded.csv")
print("NaNs después de cargar:", df.isna().sum().sum())  # También debe ser 0

NaNs después de cargar: 4426


In [7]:
df

,Name,Host Name,Host Location,Host Neighbourhood,Host Listings Count,Street,Zipcode,Market,Smart Location,Country Code,...,Bed Type_Futon,Bed Type_Pull-out Sofa,Bed Type_Real Bed,Cancellation Policy_flexible_new,Cancellation Policy_moderate,Cancellation Policy_moderate_new,Cancellation Policy_strict,Cancellation Policy_strict_new,Cancellation Policy_super_strict_30,Cancellation Policy_super_strict_60
0,The Loft-Full Bath-Deck w/View,John,My father was USAF so I was born at Offutt AFB...,NaN,1.0,"Canyon Edge Drive, Austin, TX 78733, United St...",78733,Austin,"Austin, TX",US,...,0,0,1,0,1,0,0,0,0,0
1,"Claris I, Friendly Rentals",Claudia,"Barcelona, Catalonia, Spain",NaN,162.0,"Barcelona, Barcelona 08010, Spain",08010,Barcelona,"Barcelona, Spain",ES,...,0,0,1,0,0,0,0,0,1,0
2,"Style Terrace Red, Friendly Rentals",Claudia,"Barcelona, Catalonia, Spain",NaN,162.0,"Barcelona, Barcelona 08008, Spain",08008,Barcelona,"Barcelona, Spain",ES,...,0,0,1,0,0,0,0,0,1,0
3,Picasso Suite 1.4 Paseo de Gracia,Damien,"Barcelona, Cataluña, Spain",El Born,27.0,"Barcelona, Barcelona 08009, Spain",08009,Barcelona,"Barcelona, Spain",ES,...,0,0,1,0,0,0,1,0,0,0
4,Smart City Centre Apartment II,Javier,"Madrid, Community of Madrid, Spain",L'Antiga Esquerra de l'Eixample,207.0,"L'Antiga Esquerra de l'Eixample, Barcelona, Ca...",08036,Barcelona,"Barcelona, Spain",ES,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14627,Spring into Bed Stuy!,Blair,"New York, New York, United States",Bedford-Stuyvesant,1.0,"Bedford-Stuyvesant, Brooklyn, NY 11213, United...",11213,New York,"Brooklyn, NY",US,...,0,0,1,0,0,0,1,0,0,0
14628,"Paris 15 - Chambre cosy , near Eiffel tower",Sylvia,France,Porte de Versailles,1.0,"Porte de Versailles, Paris, Île-de-France 7501...",75015,Paris,"Paris, France",FR,...,0,0,1,0,0,0,0,0,0,0
14629,Cozy Appartment Paris Batignolles,Ivan,"Madrid, Community of Madrid, Spain",NaN,1.0,"Paris, Île-de-France, France",NaN,Paris,"Paris, France",FR,...,0,0,1,0,0,0,1,0,0,0
14630,City Park West Victorian Flat,Kc,"Denver, Colorado, United States",City Park West,1.0,"Gaylord Street, Denver, CO 80206, United States",80206,Denver,"Denver, CO",US,...,0,0,1,0,1,0,0,0,0,0


In [4]:
pd.DataFrame({'columnas': df.columns})

,columnas
0,Name
1,Host Name
2,Host Location
3,Host Neighbourhood
4,Host Listings Count
5,Street
6,Zipcode
7,Market
8,Smart Location
9,Country Code


In [5]:
# === GENERACIÓN DE VARIABLES ===

# Combinar Guests Included con Availability para estimar potencial de ocupación
df['capacity_index'] = df['Guests Included'] * df['Availability 365']

# Confirmar que fue creada
print(" 'capacity_index' creada usando Guests Included × Availability 365")

 'capacity_index' creada usando Guests Included × Availability 365


In [6]:
from sklearn.model_selection import train_test_split

# Separo variables predictoras y la variable objetivo
X = df.drop(columns='Price_log')
y = df['Price_log']

# Eliminar columnas no numéricas si quedan
non_numeric_cols = X.select_dtypes(include='object').columns.tolist()
if non_numeric_cols:
    print(f"Columnas no numéricas detectadas y eliminadas: {non_numeric_cols}")
    X = X.drop(columns=non_numeric_cols)

print(f" Conjuntos creados: X_train: {X_train.shape}, X_test: {X_test.shape}")

Columnas no numéricas detectadas y eliminadas: ['Name', 'Host Name', 'Host Location', 'Host Neighbourhood', 'Street', 'Zipcode', 'Market', 'Smart Location', 'Country Code', 'Country', 'Geolocation']


NameError: name 'X_train' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f" Conjuntos creados: X_train: {X_train.shape}, X_test: {X_test.shape}")

# Entrenar modelo Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Importancia de variables
feature_importance = pd.DataFrame({
    'Variable': X_train.columns,
    'Importancia': rf.feature_importances_
}).sort_values('Importancia', ascending=False)

# Visualizar top 20
plt.figure(figsize=(12, 6))
sns.barplot(x='Importancia', y='Variable', data=feature_importance.head(20))
plt.title('Top 20 variables por importancia - Random Forest')
plt.tight_layout()
plt.show()

In [ ]:
# Establezco un umbral de importancia
umbral = 0.005

# Selecciono las variables importantes
vars_seleccionadas = feature_importance[feature_importance['Importancia'] >= umbral]['Variable'].tolist()

# Filtrar X_train y X_test
X_train_sel = X_train[vars_seleccionadas]
X_test_sel = X_test[vars_seleccionadas]

print(f" Variables seleccionadas: {len(vars_seleccionadas)}")
print(f"X_train_sel: {X_train_sel.shape}, X_test_sel: {X_test_sel.shape}")

In [ ]:
# 1. Imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 2. División train/test (estratificando en 5 bins del target para equilibrar)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=pd.qcut(y, q=5, duplicates='drop')
)

# 3. Definición de modelos dentro de pipelines
models = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LinearRegression())
    ]),
    "LassoCV": Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", LassoCV(cv=5, random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("scaler", StandardScaler()),
        ("rf", RandomForestRegressor(random_state=42))
    ]),
    "HistGradientBoosting": Pipeline([
        ("scaler", StandardScaler()),
        ("hgb", HistGradientBoostingRegressor(random_state=42))
    ])
}

# 4. Entrenamiento y evaluación en test
results = []
for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    results.append({
        "Modelo": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2": r2_score(y_test, y_pred)
    })

df_results = pd.DataFrame(results).sort_values("R2", ascending=False).reset_index(drop=True)
display(df_results)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "rf__n_estimators": [100, 300, 500, 800],
    "rf__max_depth": [None, 10, 20, 30],
    "rf__min_samples_split": [2, 5, 10],
    "rf__min_samples_leaf": [1, 2, 4],
    "rf__max_features": ["auto", "sqrt", 0.3, 0.5]
}

rs_rf = RandomizedSearchCV(
    models["RandomForest"],
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    random_state=42
)
rs_rf.fit(X_train, y_train)
print("Mejores parámetros:", rs_rf.best_params_)
print("R² CV best:", rs_rf.best_score_)

In [ ]:
# Defino modelos
modelos = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

# Cross Validation y evaluación para cada modelo
resultados = []

for nombre, modelo in modelos.items():
    print(f"\n Evaluando modelo: {nombre}")
    
    # Cross-validation (CV=5)
    cv = cross_validate(modelo, X_train_sel, y_train,
                        scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'),
                        cv=5, return_train_score=False)

    # Promedios
    r2 = np.mean(cv['test_r2'])
    rmse = -np.mean(cv['test_neg_root_mean_squared_error'])
    mae = -np.mean(cv['test_neg_mean_absolute_error'])

    resultados.append({
        'Modelo': nombre,
        'R²': r2,
        'RMSE': rmse,
        'MAE': mae
    })

    print(f"R²: {r2:.4f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}")

# Mostrar comparación
df_resultados = pd.DataFrame(resultados).sort_values('R²', ascending=False)
display(df_resultados)

In [ ]:
reg_experiment = setup(
    data=df,
    target='Price_log',
    session_id=42,
    remove_multicollinearity=True,
    multicollinearity_threshold=0.95,
    normalize=True,
    verbose=False 
)

In [ ]:
# 2. Comparar múltiples modelos automáticamente
best_model = compare_models()

# 3. Ver el modelo ganador y su rendimiento
print(best_model)

# 4. Opcional: evaluar, guardar o usar el modelo
evaluate_model(best_model)

In [ ]:
from pycaret.regression import *

# 1. Inicializa el entorno de trabajo (si no lo hiciste ya)
reg_experiment = setup(data=df, target='Price_log', session_id=42, normalize=True, verbose=False)

# 2. Crea el modelo base
rf_model = create_model('rf')

# 3. Ahora sí, lo puedes tunear
tuned_rf = tune_model(rf_model, optimize='R2')

# 4. Evaluar gráficamente
evaluate_model(tuned_rf)

# 5. Entrenar modelo final sobre todo el dataset
final_rf = finalize_model(tuned_rf)